In [74]:
import openai
import langchain
import faiss
import pandas as pd

In [56]:
import os
import faiss

import PyPDF2
import numpy as np

In [2]:
OPEN_API_KEY=""

In [57]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
from langchain.embeddings import SentenceTransformerEmbeddings
from pinecone import Pinecone

In [58]:
def extract_text_from_pdf(pdf_path):
    text = []
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        num_pages = len(reader.pages)
        print("NUM Pages:",num_pages)
        for page in range(num_pages):
            pdf_page = reader.pages[page]
            text.append(pdf_page.extract_text())
    return text


In [59]:
def embeddings_creation(data):
        chunks=data
        # List to store the embeddings
        # Generate embeddings for each chunk
        for chunk in chunks:
            paragraph.append(chunk)
            response = embedding.embed_query(chunk)
            embeddings.append(response)
        return embeddings, paragraph

In [1]:
openai.api_key=""



embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")



NameError: name 'openai' is not defined

In [62]:
embeddings = []
paragraph = []


In [63]:
def embeddings_to_indexer(embeddings):
        nlist = 1
        # Convert the embeddings to a numpy array
        embeddings = np.array(embeddings)
        # Set the dimension of the embeddings
        embedding_dim = embeddings.shape[1]
        print("Embedding Dimension:",embedding_dim)
        # print("Embedding Dimension:",embedding_dim)
        # Initialize the index
        # Add the embeddings to the index
        quantiser = faiss.IndexFlatL2(embedding_dim)
        indexer = faiss.IndexIVFFlat(quantiser, embedding_dim, nlist,   faiss.METRIC_L2)
        
        #train the model
        indexer.train(embeddings)
        indexer.add(embeddings)
        return indexer

In [64]:

def question_query(question):
        query_vector = embedding.embed_query(question)
        return query_vector

In [81]:
def question_answering_model(query_vector, paragraph,indexer):
    embedd = np.array(query_vector)
    embed_reshaped = embedd.reshape(1, 384)
    distances, indices = indexer.search(embed_reshaped,k=2)
    relevant_documents = ''

    for index in indices[0]:
        # Replace this with your actual code to retrieve the relevant documents based on the index
        document = paragraph[index]
        relevant_documents += (document + " ")

    print(relevant_documents)
    return relevant_documents

In [82]:
def make_prompt(user_query, relevant_documents):
  escaped = relevant_documents.replace("'", "").replace('"', "").replace("\n", " ")
  prompt = textwrap.dedent("""You are a helpful and informative bot that answers questions using text from the reference passage included below. \
  Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. \
  However, you are talking to a non-technical audience, so be sure to break down complicated concepts and \
  strike a friendly and converstional tone. \
  If the passage is irrelevant to the answer, you may ignore it.
  QUESTION: '{user_query}'
  PASSAGE: '{relevant_documents}'

    ANSWER:
  """).format(user_query=user_query, relevant_documents=escaped)

  return prompt

In [83]:
pdf_path = os.path.join("data", "lawsofpower.pdf")

In [84]:
data = extract_text_from_pdf(pdf_path)
embeddings, paragraph = embeddings_creation(data)
print(len(embeddings),len(paragraph))
faiss_indexer = embeddings_to_indexer(embeddings)
print("CREATED")

NUM Pages: 17
34 34
Embedding Dimension: 384
CREATED


In [85]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response =openai.chat.completions.create(
             model="gpt-3.5-turbo",
             messages=[{"role": "user", "content": prompt}]
)
    return response.choices[0].message

In [86]:
def predict(question):
    #  embeddings, paragraph = embeddings_creation(data)
     
     query_vector = question_query(question)
     relevant_documents = question_answering_model(query_vector, paragraph, indexer=faiss_indexer)
     prompt = make_prompt(query_vector,relevant_documents)
     answer=get_completion(prompt)
     return Markdown(answer.text)

In [93]:
query_vector = question_query("Can you tell me the story of Queen Elizabeth I from this 48 laws of power book?")


In [94]:
relevant_documents = question_answering_model(query_vector, paragraph, indexer=faiss_indexer)

 
 
The 48 Laws Of Power 
 
 
 
 
By  
Robert Greene 
 
 
 
 
Penguin Books 2000 
Hardcover Edition ISBN 0-670-88146-5 
Paperback Edition ISBN 0 14 02.8019 7 
452 pages 
 
 
WISDOM IN A NUTSHELL  
 
The 48 Laws Of Power 
 
 
 
 
By  
Robert Greene 
 
 
 
 
Penguin Books 2000 
Hardcover Edition ISBN 0-670-88146-5 
Paperback Edition ISBN 0 14 02.8019 7 
452 pages 
 
 
WISDOM IN A NUTSHELL 
